In [1]:
import os
import pathlib
import time

import findspark

prj_dir = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_dir / 'spark-3.5.0-bin-hadoop3')

findspark.init(spark_home)

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext


conf = SparkConf().setMaster('local[2]').setAppName('Join Operation').set('spark.executer.memory', '2g')
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/21 13:49:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Accumulator simple example

In [2]:
acc = sc.accumulator(0)
sc.parallelize([1,2,3,4]).foreach(lambda x: acc.add(x))

acc.value

10

# Exercise

In [3]:
from pyspark.accumulators import AccumulatorParam


class DiffAccumulatorParam(AccumulatorParam):

    # Built-in method  
    def zero(self, value):
        return [0.0] * len(value)

    # Built-in method  
    def addInPlace(self, val1, val2):
        for i in range(len(val1)):
            val1[i] += val2[i]
        return val1

In [4]:
va = sc.accumulator(
    value=[1.0, 2.0, 3.0],
    accum_param=DiffAccumulatorParam()
)

print(va.value)

[1.0, 2.0, 3.0]


In [5]:
rdd1 = sc.parallelize([4,5,6])

def diff(x):
    global va
    va += [x] * 3
    
rdd1.collect()

[4, 5, 6]

In [6]:
rdd1.foreach(diff)
print(va.value)

[16.0, 17.0, 18.0]
